# Projeto de Conclusão de Curso

Projeto desenvolvido para conclusão do curso Formação Power BI pela escola [Cubo Três](https://app.escoladedados.com/trilhas), ministrada pelo [Laennder Alves](https://www.linkedin.com/in/laennder/).

O [João Oliveira](https://www.linkedin.com/in/joaomdeoliveira/) nos desafiou a postar em formato de artigo.

Neste projeto, será abordado:

- A compactação de arquivos grandes para um melhor desempenho do Power BI
- Análise de dados para identificar padrões.


**Nota:** Os dados utilizados neste notebook foram retirados do [Kaggle](https://www.kaggle.com/datasets/danlessa/brazil-interstate-bus-travels%EF%BF%BC)

**Por:** [Yara Rufino](https://www.linkedin.com/in/yara-de-oliveira-rufino/)

In [9]:
#Versão das bibliotecas instaladas

#!pip install polars
#!pip show polars
#import sys
#print(sys.version)

In [11]:
#Versão da Linguagem Python
from platform import python_version
import jupyter_core

print('Versão do Python:', python_version())
print('Versão do Jupyter Notebook: ',jupyter_core.__version__)

Versão do Python: 3.11.5
Versão do Jupyter Notebook:  5.3.0


##  <font color=red> 1. Carregar dados

 Mostrar todos os arquivos simultaneamente 

In [1]:
#Bibliotecas
import polars as pl
import os

In [5]:
#Especificar o diretório onde os arquivos estão
diretorio_raiz = r'C:\Users\Usuário\Documents\Cursos\Escola de Dados\Formação PBI\Mod8'

#Lista para armazenar os caminhos dos arquivos CSV encontrados
arquivos_csv = []

#Percorrer o diretório raiz e subdiretórios procurando por arquivos CSV
for pasta_raiz, subpastas, arquivos in os.walk(diretorio_raiz):
    for arquivo in arquivos:
        if arquivo.endswith('.csv'):
            caminho_completo = os.path.join(pasta_raiz, arquivo)
            arquivos_csv.append(caminho_completo)

#Arquivos CSV encontrados
for arquivo_csv in arquivos_csv:
    print(arquivo_csv)

C:\Users\Usuário\Documents\Cursos\Escola de Dados\Formação PBI\Mod8\venda_passagem_01_2020\venda_passagem_01_2020.csv
C:\Users\Usuário\Documents\Cursos\Escola de Dados\Formação PBI\Mod8\venda_passagem_03_2019\venda_passagem_03_2019.csv
C:\Users\Usuário\Documents\Cursos\Escola de Dados\Formação PBI\Mod8\venda_passagem_11_2019\venda_passagem_11_2019.csv
C:\Users\Usuário\Documents\Cursos\Escola de Dados\Formação PBI\Mod8\venda_passagem_12_2019\venda_passagem_12_2019.csv


In [6]:
#Criar uma lista para armazenar os DataFrames de cada arquivo
dfs = []
chunk_size = 100000 #parte do arq que é processada por vez

#Loop pelos arquivos e ler cada um em um DataFrame
for arquivo_csv in arquivos_csv:
    #Ler o arquivo CSV usando o Polars
    dtypes = {"valor_percentual_desconto": str} #converter em string pois como int ele apresenta erro
    df = pl.read_csv(arquivo_csv, has_header=True, schema=None, truncate_ragged_lines=True, separator=';', dtypes=dtypes,
                     batch_size=chunk_size, infer_schema_length=100000, encoding='latin-1')

    dfs.append(df)

#Exibindo todos os dataframes
print(dfs)

[shape: (3_030_932, 26)
┌──────────────┬──────────────┬──────────────┬─────────────┬───┬─────────────┬─────────────┬─────────────┬─────────────┐
│ codigo_viage ┆ cnpj         ┆ numero_equip ┆ numero_bilh ┆ … ┆ valor_taxa_ ┆ valor_total ┆ origem_emis ┆ in_passagem │
│ m            ┆ ---          ┆ amento_fisca ┆ ete         ┆   ┆ embarque    ┆ ---         ┆ sao         ┆ _cancelada  │
│ ---          ┆ i64          ┆ l            ┆ ---         ┆   ┆ ---         ┆ f64         ┆ ---         ┆ ---         │
│ i64          ┆              ┆ ---          ┆ i64         ┆   ┆ f64         ┆             ┆ str         ┆ str         │
│              ┆              ┆ str          ┆             ┆   ┆             ┆             ┆             ┆             │
╞══════════════╪══════════════╪══════════════╪═════════════╪═══╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 2486967      ┆ 300693140001 ┆ 351912300693 ┆ 370123      ┆ … ┆ 6.39        ┆ 123.99      ┆ Não Se      ┆ NÃO         │
│       

In [7]:
#Tempo de execução
%time dfs = pl.read_csv(caminho_completo, separator=';', ignore_errors = True, schema=None, truncate_ragged_lines=True)

CPU times: total: 23.3 s
Wall time: 21 s


## <font color=red> 2. Converter no formato parquet
Processo de compactção dos arquivos

In [8]:
#Caminho para o novo diretório onde os arquivos Parquet serão salvos
diretorio_parquet = r'C:\Users\Usuário\Documents\Cursos\Escola de Dados\Formação PBI\Mod8\Compactado'

#Criar o novo diretório se não existir
os.makedirs(diretorio_parquet, exist_ok=True)

#Loop pelos arquivos csv e convertê-los para Parquet
for arquivo_csv in arquivos_csv:
    #Ler o arquivo de texto em um DataFrame Polars
    dtypes = {"valor_percentual_desconto": str}
    df = pl.read_csv(arquivo_csv, has_header=True, schema=None, truncate_ragged_lines=True, separator=';', dtypes=dtypes,
                     batch_size=chunk_size, infer_schema_length=100000, encoding='latin-1')
    #Extrair o nome do arquivo sem a extensão
    nome_arquivo = os.path.splitext(os.path.basename(arquivo_csv))[0]
    
    #Caminho completo para o arquivo Parquet
    caminho_arquivo_parquet = os.path.join(diretorio_parquet, f'{nome_arquivo}.parquet')
    
    #Salvar o DataFrame convertido em Parquet
    df.write_parquet(caminho_arquivo_parquet)

In [9]:
#Conferir se salvou todos os arquivos
diretorio_parquet =  r'C:\Users\Usuário\Documents\Cursos\Escola de Dados\Formação PBI\Mod8\Compactado'

arquivos_pqt = [arquivo for arquivo in os.listdir(diretorio_parquet) if arquivo.endswith('.parquet')]
print(arquivos_pqt)

['venda_passagem_01_2020.parquet', 'venda_passagem_03_2019.parquet', 'venda_passagem_11_2019.parquet', 'venda_passagem_12_2019.parquet']


In [10]:
#Leitura dos arquivos compactados
dfs = []

for arquivo_pqt in arquivos_pqt:
    caminho_arquivo_pqt = os.path.join(diretorio_parquet, arquivo_pqt)
    df = pl.read_parquet(caminho_arquivo_pqt)
    dfs.append(df)
    
display(dfs)

[shape: (3_030_932, 26)
 ┌──────────────┬──────────────┬──────────────┬─────────────┬───┬─────────────┬─────────────┬─────────────┬─────────────┐
 │ codigo_viage ┆ cnpj         ┆ numero_equip ┆ numero_bilh ┆ … ┆ valor_taxa_ ┆ valor_total ┆ origem_emis ┆ in_passagem │
 │ m            ┆ ---          ┆ amento_fisca ┆ ete         ┆   ┆ embarque    ┆ ---         ┆ sao         ┆ _cancelada  │
 │ ---          ┆ i64          ┆ l            ┆ ---         ┆   ┆ ---         ┆ f64         ┆ ---         ┆ ---         │
 │ i64          ┆              ┆ ---          ┆ i64         ┆   ┆ f64         ┆             ┆ str         ┆ str         │
 │              ┆              ┆ str          ┆             ┆   ┆             ┆             ┆             ┆             │
 ╞══════════════╪══════════════╪══════════════╪═════════════╪═══╪═════════════╪═════════════╪═════════════╪═════════════╡
 │ 2486967      ┆ 300693140001 ┆ 351912300693 ┆ 370123      ┆ … ┆ 6.39        ┆ 123.99      ┆ Não Se      ┆ NÃO         │


In [16]:
#Tempo de execução
%time dfs = pl.read_csv(caminho_completo, separator=';', ignore_errors = True, schema=None, truncate_ragged_lines=True)

CPU times: total: 13.4 s
Wall time: 5.58 s
